In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

Initializing files

In [2]:
EmissionsPerCar = pd.read_csv('CO2_Emissions.csv')

In [6]:
CarRating = pd.DataFrame(columns= ['Make', 'Model', 'max', 'Rating'])
CarRating['Make'] = EmissionsPerCar['Make']
CarRating['Model'] = EmissionsPerCar['Model']

CarRating['Rating'] = 10 - (EmissionsPerCar['CO2 Emissions(g/km)']/52.2)
max_rating = CarRating.loc[CarRating['Rating'].idxmax()]
min_rating = CarRating.loc[CarRating['Rating'].idxmin()]
AverageRating = CarRating['Rating'].mean()

print("The car with the best rating is a " + str(max_rating['Make']) +" " + str(max_rating['Model']) + " with a rating of " + str(max_rating['Rating']))
print("The car with the worst rating is "+ str(min_rating['Make']) +" " + str(min_rating['Model']) + " with a rating of " + str(min_rating['Rating']))
print("The average rating of the 7200 car models is " + str(AverageRating))


The car with the best rating is a HYUNDAI IONIQ BLUE with a rating of 8.160919540229886
The car with the worst rating is BUGATTI CHIRON with a rating of 0.0
The average rating of the 7200 car models is 5.1995268445668845
